In [1]:
import pandas as pd
import numpy as np
from time import time
from scipy.optimize import minimize, Bounds, LinearConstraint, NonlinearConstraint
from functools import partial

In [2]:
# current_volume = 75.3e6 # database
# growth_ambition_perc = 0.1 ## user_input
# growth_ambition_volume = current_volume*growth_ambition_perc

remainder_volume_growth = 124567 # growth_ambition_volume - achieved_from_media

In [3]:
data = pd.read_excel("./test_data/KSA Pepsi.xlsx", 
                     sheet_name="distribution_records v2", engine="openpyxl")
data.tail()

,id,country,channel,current_metrics_time,current_volume_time,analysis_period,category,brand,currency,volume_unit,...,current_volume,price_elasticity,current_price_per_volume,current_price_per_pack,current_volume_per_pack,distribution_type,current_distribution,distribution_elasticity,current_trade,trade_elasticity
43,44,KSA,Traditional Trade,2021-05-01,CY 2020,Jan 2018 to May 2021,Beverage,Pepsi,SAR,8Oz case,...,35295,-3.907624,43.755533,2.774088,0.06340,TDP,6.3115,0.357306,NaN,NaN
44,45,KSA,Traditional Trade,2021-05-01,CY 2020,Jan 2018 to May 2021,Beverage,Pepsi,SAR,8Oz case,...,2970959,0.000000,28.396171,5.001132,0.17612,TDP,91.1393,1.297495,NaN,NaN
45,46,KSA,Traditional Trade,2021-05-01,CY 2020,Jan 2018 to May 2021,Beverage,Pepsi,SAR,8Oz case,...,997850,-1.699106,23.227203,8.999611,0.38746,TDP,56.9799,0.000000,NaN,NaN
46,47,KSA,Traditional Trade,2021-05-01,CY 2020,Jan 2018 to May 2021,Beverage,Pepsi,SAR,8Oz case,...,9502406,-1.699106,22.708695,8.998774,0.39627,TDP,84.3787,0.000000,NaN,NaN
47,48,KSA,Traditional Trade,2021-05-01,CY 2020,Jan 2018 to May 2021,Beverage,Pepsi,SAR,8Oz case,...,22547413,NaN,32.148220,NaN,NaN,NaN,NaN,NaN,0.047549,0.126164


In [4]:
def volume_growth(input_val, current_val, current_volume, elasticity, return_sum=False):
#     if current_val == 0: return 0
    pct_change_lever = input_val/current_val - 1
    pct_change_volume = pct_change_lever*elasticity
    vg = pct_change_volume*current_volume
    if return_sum: return sum(vg)
    return vg

def cost_of_distribution(input_val, current_val, current_volume):
    K = 0.01
    cost = K*(input_val - current_val)
    return sum(cost)

def cost_of_price(input_val, current_val, current_volume, elasticity):
    vg = volume_growth(input_val, current_val, current_volume, elasticity)
    cost = vg*(input_val - current_val)
    return sum(cost)

def cost_of_trade(input_val, current_val, current_volume, elasticity, current_price_per_volume):
    vg = volume_growth(input_val, current_val, current_volume, elasticity)
    cost = vg*(input_val - current_val)*current_price_per_volume
    return sum(cost)


def cost(x, lever, current_val, current_volume, elasticity, current_price_per_volume, bounds=None):
    if bounds is not None: 
        if not all([z[0] <= z[1] <= z[2] for z in zip(bounds.lb, x, bounds.ub)]): 
            return np.inf # fix
    
    price_idx = np.where(lever == 'price')[0].tolist()
    distribution_idx = np.where(lever == 'distribution')[0].tolist()
    trade_idx = np.where(lever == 'trade')[0].tolist()
    
    p = cost_of_price(x[price_idx], current_val[price_idx], current_volume[price_idx], elasticity[price_idx])
    d = cost_of_distribution(x[distribution_idx], current_val[distribution_idx], current_volume[distribution_idx])
    t = cost_of_trade(x[trade_idx], current_val[trade_idx], current_volume[trade_idx], elasticity[trade_idx], current_price_per_volume[trade_idx])
    
    cost = p + d + t
    return cost

In [5]:
def hessian(x, *args):
    return np.zeros((x.shape[0], x.shape[0]))

def volume_constraint(current_val, current_volume, elasticity, x):
    vg = volume_growth(x, current_val, current_volume, elasticity, return_sum=True)
    return vg

In [6]:
idx = ['id', 'id', 'id']
lever = ['price', 'distribution', 'trade']
current_val = ['current_price_per_pack', 'current_distribution', 'current_trade']
current_volume = ['current_volume', 'current_volume', 'current_volume']
elasticity = ['price_elasticity', 'distribution_elasticity', 'trade_elasticity']
current_price_per_volume = ['current_price_per_volume', 'current_price_per_volume', 'current_price_per_volume']

dfs = []
for i, e in enumerate(elasticity):
    df = pd.DataFrame(columns=['id', 'lever', 'current_val', 'current_volume', 'elasticity', 'current_price_per_volume'])
    sub_data = data.dropna(subset=[e])
    df['id'] = sub_data[idx[i]]
    df['lever'] = lever[i]
    df['current_val'] = sub_data[current_val[i]]
    df['current_volume'] = sub_data[current_volume[i]]
    df['elasticity'] = sub_data[elasticity[i]]
    df['current_price_per_volume'] = sub_data[current_price_per_volume[i]]
    dfs.append(df.reset_index(drop=True))
    
df = pd.concat(dfs, axis=0)
assert df.isna().sum().sum() == 0

In [7]:
### use user input for trade, use different B for P|D

B = 0.10
df['constraint_lower'] = df['current_val']*(1-B)
df['constraint_upper'] = df['current_val']*(1+B)

In [8]:
bounds = Bounds(lb=df['constraint_lower'].values, 
                ub=df['constraint_upper'].values, 
                keep_feasible=False)
nl_constraints = NonlinearConstraint(fun=partial(volume_constraint, df['current_val'].values, df['current_volume'].values, df['elasticity'].values),
                                   lb=remainder_volume_growth, ub=np.inf, keep_feasible=False)
x0  = df['current_val'].values
iterations = int(1e3)

In [9]:
stime = time()
result = minimize(cost, x0 = x0, 
                  args = (df['lever'].values, df['current_val'].values, df['current_volume'].values, df['elasticity'].values, df['current_price_per_volume'].values, bounds), 
                  bounds = bounds, method = 'trust-constr', constraints = [nl_constraints],
                  hess=hessian, options = {'maxiter': iterations, 'verbose': 1, 'factorization_method':'SVDFactorization'}
                 )
                  
print(f"Ran in {(time()-stime)/60:.1f}m")
print(result.success, result.cg_stop_cond)

`xtol` termination condition is satisfied.
Number of iterations: 301, function evaluations: 11455, CG iterations: 289, optimality: 1.54e+05, constraint violation: 0.00e+00, execution time:  3.7 s.
Ran in 0.1m
True 2


In [11]:
vg = volume_growth(result.x, df['current_val'].values, df['current_volume'].values, df['elasticity'].values, return_sum=True)
print(f"Volume Achieved: {int(vg):,} 8Oz cases")

Volume Achieved: 124,568 8Oz cases


In [14]:
c = cost(result.x, lever, df['current_val'].values, df['current_volume'].values, df['elasticity'].values, df['current_price_per_volume'].values)
print(f"At the cost of: SAR {int(c):,} ")

At the cost of: SAR 0 


In [12]:
if result.success:
    df['recommendation'] = result.x
    df['volume_growth'] = volume_growth(result.x, df['current_val'].values, df['current_volume'].values, df['elasticity'].values, return_sum=False)
    df.to_csv("./test_data/KSA Pepsi Execution Out.csv", index=False)